In [1]:
# this shall train neural network to recognize category / words ( total 30 ) from 1 second audio
# input audio format : format 16-bit little-endian PCM-encoded WAVE file at a 16000 sample rate

# to do : deal with padding and batching , deal with audio process / dataset , loss function

import tensorflow as tf
from __future__ import print_function
from tensorflow.contrib import rnn
import scipy.io.wavfile as wav
from python_speech_features import mfcc
import numpy as np


In [2]:
def audio_to_mfcc(fileurl):
    rate, sig = wav.read(fileurl)
    mfcc_feat = mfcc(sig,rate)
    #d_mfcc_feat = delta(mfcc_feat, 2)
    #fbank_feat = logfbank(sig,rate)

    return mfcc_feat

In [4]:
# test above function
inputs = audio_to_mfcc('/home/saurabh/Documents/tf_orange/tf_orange/data/test.wav')

In [5]:
# Training Parameters
learning_rate = 0.001
training_steps = 20
batch_size = 2
display_step = 200


# Network Parameters

num_features = 13

timesteps = 300 # maximum timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 30   # since there are 30 categories 

In [6]:
# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_features])
Y = tf.placeholder("float", [None, num_classes])

In [7]:
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([batch_size, 2 * num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [8]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell_fw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
    lstm_cell_bw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    
    # Get lstm cell output
    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_cell_fw, lstm_cell_bw, x,
    dtype=tf.float32)
    
    # get ony the output of last rnn timestep
   # outputs=outputs[-1]

    outputs=tf.transpose( outputs , [1, 0, 2])
    
    
    
    
    
  
   # res =  tf.matmul(outputs, weights['out']) + biases['out']
    
    return tf.nn.softmax(tf.matmul(outputs, weights['out']) + biases['out'])[-1]

In [9]:
inputs = tf.placeholder(tf.float32, [None, None, num_features])

In [10]:
logits = RNN(inputs, weights, biases)

In [11]:
init = tf.global_variables_initializer()

In [12]:
with tf.Session() as sess:
    

    # Run the initializer
    sess.run(init)
    
    feed = { inputs : np.random.rand(2, 300 , 13) }
    
    print(sess.run(logits,feed  ))

[[ 0.05123276  0.03327633  0.00066453 ...,  0.02221111  0.04170784
   0.01102589]
 [ 0.02923222  0.04898273  0.000432   ...,  0.04249125  0.03267223
   0.00784348]
 [ 0.02753832  0.05062243  0.00049461 ...,  0.05555979  0.02003477
   0.00913165]
 ..., 
 [ 0.02658585  0.03715054  0.00101787 ...,  0.04007464  0.00781681
   0.00587208]
 [ 0.03315795  0.03975964  0.00222904 ...,  0.02789835  0.01048326
   0.00665665]
 [ 0.03876461  0.02740019  0.00347753 ...,  0.0224323   0.01012829
   0.00967273]]
